# CARTHE Glad Experiment

To process the [CARTHE GLAD](http://carthe.org/glad/) experiment dataset, we follow the same steps described in `dataformat-gdp.ipynb`. The only difference is that we defined a new preprocessing functions (`data/glad.py`) tailored to the format of the individual NetCDF files for this dataset.

In [1]:
import sys
sys.path.insert(0, '../')
from clouddrift.dataformat import ragged_array

## Dataset-specific functions

For this dataset, importing the glad module *automatically* download the unique ASCII file `GLAD_15min_filtered.dat` and stored it into the `data/raw/glad/` folder. Subsequently, the dataset is parse using pandas.

- `glad.preprocess`: applies preprocessing routine and return an `xarray.Dataset` for a specific trajectory

In [2]:
from data import glad

Dataset GLAD_15min_filtered.dat already in '../data/raw/glad/'.


We can test the preprocessing function by calling it with one of the identification numbers:

In [3]:
glad.preprocess(1)

<xarray.Dataset>
Dimensions:    (traj: 1, obs: 7695)
Coordinates:
    longitude  (obs) float64 -87.21 -87.21 -87.21 ... -84.61 -84.62 -84.62
    latitude   (obs) float64 28.56 28.56 28.56 28.56 ... 29.1 29.1 29.1 29.1
    time       (obs) datetime64[ns] 2012-07-20T01:30:00.172800 ... 2012-10-08...
    ids        (obs) int64 1 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
Dimensions without coordinates: traj, obs
Data variables:
    ID         (traj) int64 1
    rowsize    (traj) int64 7695
    err_pos    (obs) float64 10.0 10.0 10.0 10.0 10.0 ... 13.2 13.2 13.0 12.8
    ve         (obs) float64 0.022 0.021 0.021 0.022 ... -0.082 -0.077 -0.071
    vn         (obs) float64 -0.23 -0.213 -0.197 -0.184 ... -0.052 -0.052 -0.05
    err_vel    (obs) float64 0.033 0.033 0.033 0.033 ... 0.033 0.033 0.038 0.055
Attributes:
    title:    Glad experiment

It is now possible to create the ragged array and either save a NetCDF file, a parquet file, or simply output an Awkward Array that can be used for analysis:

In [4]:
files = [  1,   2,   3,   4,   5,   6,   7,   8,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        83,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,
       188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,
       201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
       214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252,
       253, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279,
       280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 301, 302,
       303, 304, 306, 307, 308, 310, 313, 314, 315, 317, 451]

coords = {'ids': 'ids', 'time': 'time', 'lon': 'longitude', 'lat': 'latitude'}
metadata = ['ID', 'rowsize']
data = ['ve', 'vn', 'err_pos', 'err_vel']

ra = ragged_array.from_files(
    files,
    glad.preprocess,
    coords,
    metadata,
    data
)

Filling the Ragged Array: 100%|█████████████| 297/297 [00:00<00:00, 1108.82it/s]


## Export to data files:

In [5]:
ra.to_parquet('../data/process/glad.parquet')
ra.to_netcdf('../data/process/glad.nc')

## Import from data files:

In [7]:
ra2 = ragged_array.from_netcdf('../data/process/glad.nc')

In [8]:
# this function not working for Shane
ra2 = ragged_array.from_parquet('../data/process/glad.nc')

/Users/selipot/opt/miniconda3/envs/clouddrift/lib/python3.9/site-packages/fsspec/parquet.py:225: UserWarning: Not enough data was used to sample the parquet footer. Try setting footer_sample_size >= 969367282.
  warnings.warn(


ArrowInvalid: Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

## Convert to Awkward Array:

In [10]:
ds = ra.to_awkward()

In [11]:
ds.ID

<Array [1, 2, 3, 4, 5, ..., 314, 315, 317, 451] type='297 * int64[parameter...'>

In [12]:
ds.fields

['ID', 'rowsize', 'obs']

In [13]:
ds.obs.fields

['ids', 'time', 'lon', 'lat', 've', 'vn', 'err_pos', 'err_vel']